In [6]:
import findspark
findspark.init()
 
from pyspark.sql import SparkSession
 
spark = SparkSession.builder \
    .appName("MySparkApp") \
    .getOrCreate()
 
spark

In [7]:
ecommerceDF = spark.read.option("header", "true").option("inferSchema", "true").csv("C:/Users/shemantk/OneDrive - Capgemini/Documents/SPRINT Ecommerce Data/5000 Sales Records.csv")

In [8]:
ecommerceDF.printSchema()
ecommerceDF.show()

root
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Item Type: string (nullable = true)
 |-- Sales Channel: string (nullable = true)
 |-- Order Priority: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Order ID: integer (nullable = true)
 |-- Ship Date: string (nullable = true)
 |-- Units Sold: integer (nullable = true)
 |-- Unit Price: double (nullable = true)
 |-- Unit Cost: double (nullable = true)
 |-- Total Revenue: double (nullable = true)
 |-- Total Cost: double (nullable = true)
 |-- Total Profit: double (nullable = true)

+--------------------+--------------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|              Region|             Country|      Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|
+--------------------+

### 8.Display the number of orders for each item.

In [9]:
# Load data into a DataFrame (assuming you've already loaded your data)
df = spark.read.option("header", "true").csv("C:/Users/shemantk/OneDrive - Capgemini/Documents/SPRINT Ecommerce Data/5000 Sales Records.csv")

 
# Group by Item Type and count number of orders
item_order_counts = df.groupBy("Item Type").count()
 
# Rename the count column for clarity
item_order_counts = item_order_counts.withColumnRenamed("count", "Number of Orders")
 
# Show the result
item_order_counts.show()

# Write the result to Parquet
item_order_counts.write.mode("overwrite").parquet("C:/Users/shemantk/OneDrive - Capgemini/Documents/SPRINT Ecommerce Data/itemOrder_parquet")

+---------------+----------------+
|      Item Type|Number of Orders|
+---------------+----------------+
|      Baby Food|             445|
|         Cereal|             385|
|           Meat|             399|
|      Household|             424|
|     Vegetables|             410|
|      Beverages|             447|
|Office Supplies|             420|
|      Cosmetics|             424|
|  Personal Care|             415|
|         Fruits|             447|
|         Snacks|             398|
|        Clothes|             386|
+---------------+----------------+



### 9.Display the country with highest sale.

In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, desc

# Create Spark session
spark = SparkSession.builder.appName("DistinctCountries").master("local[*]").getOrCreate()

# Read the CSV file into DataFrame
salesDF = spark.read.option("header", "true").csv(r"C:\Users\shemantk\OneDrive - Capgemini\Documents\SPRINT Ecommerce Data\5000 Sales Records.csv")

# Coalesce to a single partition (optional)
salesDF1 = salesDF.coalesce(1)

# Group by 'Country' and sum 'Total Revenue'
countrySalesDF = salesDF1.groupBy("Country").agg(sum("Total Revenue").alias("Total_Sales")).persist()

# Find the country with the highest total sales
topCountryDF = countrySalesDF.orderBy(desc("Total_Sales")).limit(1)

# Show the result
topCountryDF.show()

# Write the result to Parquet
topCountryDF.write.mode("overwrite").parquet("C:/Users/shemantk/OneDrive - Capgemini/Documents/SPRINT Ecommerce Data/topCountry_parquet")

# Stop Spark session
spark.stop()

+-------+-------------------+
|Country|        Total_Sales|
+-------+-------------------+
| Rwanda|6.039873958999999E7|
+-------+-------------------+

